# Summarizing meeting notes

Welcome to Cohere! In this notebook, you'll learn how to:
* Use Cohere's Command-R model to summarize meeting transcripts.
* Modify your prompt to include specific formatting instructions, especially if the model output is used in downstream applications.
* Use Command-R with LangChain for summarization.

## Setup

You'll need a Cohere API key to run this notebook. If you don't have a key, head to https://cohere.com/ to generate your key.

In [1]:
%%capture
!pip install cohere datasets tokenizers langchain

In [2]:
import re
from string import Template
from typing import Optional

import cohere
from getpass import getpass
from datasets import load_dataset

# Set up Cohere client
co_api_key = getpass("Enter your Cohere API key: ")
co_model = "command-r"
co = cohere.Client(api_key=co_api_key)

Enter your Cohere API key: ··········


In [3]:
# We'll also defining some util functions for later

def pprint(s: Optional[str] = None, maxchars: int = 100):
  """
  Wrap long text into lines of at most `maxchars` (preserves linebreaks occurring in text)
  """
  if not s:
    print()
  else:
    print("\n".join(line.strip() for line in re.findall(rf".{{1,{maxchars}}}(?:\s+|$)", s)))


## Load test data

Let's load a meeting transcript to see Command in action!

* If you have your own transcript, you can load it to Colab using your favorite method.
* If you don't, we'll use a sample from the [QMSum dataset](https://github.com/Yale-LILY/QMSum). QMSum contains cleaned meeting transcripts with [diarised speakers](https://en.wikipedia.org/wiki/Speaker_diarisation).
* We'll see later that the recipe shared herein isn't limited to meeting notes transcript, but extends to any data with diarised speakers!

In [4]:
# If you have your own transcript you would like to test Command on, load it here
# transcript = ...

# Otherwise, we'll use QMSum
# Note this will download the QMSum dataset to your instance
qmsum = load_dataset("MocktaiLEngineer/qmsum-processed")
# Pick any one transcript
transcripts = qmsum["validation"]["meeting_transcript"]
transcript = transcripts[0]
pprint(transcript)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

PhD D: Sure because I I need a lot of time to to put the label or to do that
Professor E: I mean we we know that there s noise There s there s continual noise from fans and so
forth and there is more impulsive noise from taps and so forth and and something in between with
paper rustling We know that all that s there and it s a g worthwhile thing to study but obviously it
takes a lot of time to mark all of these things Whereas th i I would think that you we can study
more or less as a distinct phenomenon the overlapping of people talking So Then you can get the Cuz
you need If it s three hundred i i it sounds like you probably only have fifty or sixty or seventy
events right now that are really And and you need to have a lot more than that to have any kind of
even visual sense of of what s going on much less any kind of reasonable statistics
PhD C: Now why do you need to mark speaker overlap by hand if you can infer it from the relative
energy in the
Grad G: Well that s That s what I wa

If you've loaded the QMSum dataset, you'll see a back-and-forth dialogue between a Project Manager, an Industrial Designer, and someone responsible for the User Interface. They discuss the design of a banana-shaped remote controller.

Let's also check how many tokens that transcript corresponds to.

In [5]:
# Measure the number of tokens
num_tokens = len(co.tokenize(transcript, model=co_model).tokens)
pprint(f"Number of tokens: {num_tokens}")

Number of tokens: 1152


## Simple summarization

First, we'll show an example of a simple summarization task. The prompt template we are using is shown below, and we can obtain the completion using a one line call to the Cohere API.

In [6]:
prompt_template = Template("""## meeting transcript
$transcript

## instructions
Generate a summary of the above meeting transcript. \
Don't include preambles, postambles or explanations. \
""")
prompt = prompt_template.substitute(transcript=transcript)

# Use the Cohere API to get the response
resp = co.chat(message=prompt, model=co_model, temperature=0.2).text
pprint(resp)

The team discusses the time-consuming process of identifying instances of speaker overlap in audio
recordings. PhD C suggests using close-talking microphones and automated methods to establish ground
truth data, which Grad G is working on. This data would help develop a more efficient system than
manual marking.


We applied a few best practices in constructing this prompt:

1. We include a preamble imbuing the model with a persona.
2. We use Markdown-style headers (i.e. with `##`) to delineate the preamble, the text-to-summarise, the instructions, and the output

For some other prompting best practices and tips/tricks, see our [prompting guide](https://docs.google.com/document/d/1k0KBiOUEPq65vqEuAQyg6Y5l1s2_Ybwv-lOLho6NHsw/edit)!

## More advanced formatting

That worked well, but what if you want to have more control over the summary? What if you wanted a bulleted summary and wanted exactly 3 bullets in a json structure?

Let's try adding in some more specific formatting instructions.


In [7]:
prompt_template = Template("""## meeting transcript
$transcript

## instructions
Generate a summary of the above meeting transcript in 3 bullets. \
Use the following JSON format:
{
    "summary_bullets": [
        "<bullet 1>",
        "<bullet 2>",
        ...
    ]
}
Don't include preambles, postambles or explanations.\
""")
prompt = prompt_template.substitute(transcript=transcript)

# Use the Cohere API to get the response
resp = co.chat(message=prompt, model=co_model, temperature=0.2).text
print(resp)

```json
{
    "summary_bullets": [
        "There's a need to identify instances of speaker overlap in recorded meetings, but it's a time-consuming task to mark these events manually.",
        "An alternative is to infer speaker overlap using various measurements of energy in the audio, such as close-talking microphones, LPC residuals, etc.",
        "A threshold-based program that detects speech based on volume can speed up the process, and further improvements can be explored once a robust system is in place."
    ]
}
```


That worked well! Asking for a structured response like a JSON object is a great approach for use cases where you will need to parse or post-process the output for use in downstream applications. To ensure the best possible completion, I provided a template of the expected JSON that the model filled out.

Let's also try asking for a short 1 sentence summary.

In [8]:
prompt_template = Template("""## meeting transcript
$transcript

## instructions
Generate a summary of the above meeting transcript in 1 concise sentence. Make sure the sentence is extremely short.\
Don't include preambles, postambles or explanations.\
""")
prompt = prompt_template.substitute(transcript=transcript)

# Use the Cohere API to get the response
resp = co.chat(message=prompt, model=co_model, temperature=0.2).text
pprint(resp)

The team discusses using various methods, including close-talking mics and threshold-based volume
filters, to accurately identify instances of speaker overlap in recorded meetings.


## LangChain summarization
It's also very easy to use Command-R with LangChain for summarization.

In [9]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from langchain_community.chat_models import ChatCohere

llm = ChatCohere(
    cohere_api_key=co_api_key,
    model="command-r",
    temperature=0.2,
  )

# Define prompt
prompt_template = """## meeting transcript
{transcript}

## instructions
Generate a summary of the above meeting transcript. \
Don't include preambles, postambles or explanations."""
prompt = PromptTemplate.from_template(prompt_template)

# Define LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define StuffDocumentsChain
stuff_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="transcript", verbose=True)

# Load documents
docs = [Document(page_content=transcript)]

pprint(stuff_chain.run(docs))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new StuffDocumentsChain chain...

> Finished chain.
The team discusses the time-consuming process of identifying instances of speaker overlap in audio
recordings. PhD C suggests using close-talking microphones and automated methods to establish ground
truth data, which Grad G is working on. This data would help develop a more efficient system than
manual marking.


Command-R has a context window of 128K tokens, so you can use very long documents, or multiple documents, all in a single API call.



## Wrap up

Those were some simple examples of how you can use Cohere's Command-R model for summarization.

For more advanced summarization objectives and to see some other cool things you can do with the Command-R model, see [recipes for better meeting notes summaries](https://colab.research.google.com/drive/1XqRpJH7qRnRTbOEt6kthwqZG6gtEn4gN#scrollTo=LplM9PSe8djM).

In cases where you can't fit the input text into the context window, see our [cookbook for long document strategies](https://colab.research.google.com/drive/1zxSAbruOWwWJHNsj3N56uxZtUeiS7Evd#scrollTo=4JgYYTg7Shvq).

For more information, you can reach out to us at summarize@cohere.com!